# Text-Moderation System with Embeddings

In [ ]:
%pip install -q sentence-transformers scikit-learn pandas matplotlib

**Import core modules**

We import everything the notebook will need.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, RocCurveDisplay
import pandas as pd
import matplotlib.pyplot as plt

**Load and inspect data**

Our dataset, moderation_dataset.csv, contains two columns:

In [ ]:
df = pd.read_csv("data/moderation_dataset.csv")
df.head(15)

Let’s check the label distribution:

In [ ]:
df.label.value_counts()

Our dataset is highly imbalanced, though it includes a large number of minority class examples. In this example, we'll apply balanced sampling to train on an even class distribution.

In [ ]:
df = pd.concat((
    df[df.label == 1].sample(500),
    df[df.label == 0].sample(500)
), axis=0)

# setting up environment

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env
load_dotenv()

# Access the API key
api_key = os.getenv("OPENAI_API_KEY")

**Create sentence embeddings**

We will use OpenAI’s text-embedding-3-small model with 256 dimensions. This offers high-quality multilingual embeddings in a compact size suitable for low-latency tasks.

In [ ]:
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(api_key=api_key)

embeddings = []
for text in tqdm(df["text"].tolist(), desc="Embedding texts"):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small",
        dimensions=256
    )
    embeddings.append(response.data[0].embedding)

**Prepare train and test sets**

A stratified split keeps the class ratio consistent.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    embeddings,
    df["label"],
    test_size=0.20,
    stratify=df["label"],
    random_state=42
)

**Train a baseline classifier**

Logistic regression handles dense embeddings well and trains in seconds

In [ ]:
clf = LogisticRegression(
    max_iter=200,
    class_weight="balanced"  # helps if classes are uneven
)
clf.fit(X_train, y_train)

**Evaluate initial performance**

We first look at metrics using the default 0.5 threshold.

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))